In [ ]:
import os
import shutil
from PIL import Image
import numpy as np
import cv2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix


# Path ke dataset gambar pisang
base_dir = 'test'  # Ganti dengan path dataset gambar pisang Anda
categories = ['overripe', 'ripe', 'rotten', 'unripe']

try:
    # Membuat folder untuk setiap kategori jika belum ada
    for category in categories:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)
except Exception as e:
    print(f"Error creating directories: {e}")

# Fungsi untuk mendapatkan label dari setiap gambar
def get_label(image_path):
    # Implementasikan logika pelabelan di sini
    # Contoh berdasarkan nama folder atau kriteria lain
    if 'overripe' in image_path:
        return 'overripe'
    elif 'ripe' in image_path:
        return 'ripe'
    elif 'rotten' in image_path:
        return 'rotten'
    elif 'unripe' in image_path:
        return 'unripe'
    else:
        return 'unknown'

# Fungsi untuk menghitung nilai modus dari array numpy
def find_mode(np_array):
    vals, counts = np.unique(np_array, return_counts=True)
    index = np.argmax(counts)
    return vals[index]

# Melabeli gambar dan melakukan crop
mode_hue_values = []
labels = []
try:
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                file_path = os.path.join(root, file)
                label = get_label(file_path)
                print(f'Processing file: {file_path}, Label: {label}')
                if label != 'unknown':
                    # Pindahkan file ke folder yang sesuai
                    new_path = os.path.join(base_dir, label, file)
                    shutil.move(file_path, new_path)
                    print(f'Moved {file} to {label} folder')

                    # Crop gambar menjadi 20x20 px
                    img = Image.open(new_path)
                    img = img.resize((20, 20), Image.LANCZOS)
                    img.save(new_path)
                    print(f'Cropped {file} to 20x20 px')

                    # Normalisasi RGB
                    img_array = np.array(img)
                    img_array = img_array / 255.0
                    img_normalized = Image.fromarray((img_array * 255).astype(np.uint8))
                    img_normalized.save(new_path)
                    print(f'Normalized RGB values for {file}')

                    # Konversi RGB menjadi HSV
                    img_hsv = cv2.cvtColor(np.array(img_normalized), cv2.COLOR_RGB2HSV)
                    img_hsv = Image.fromarray(img_hsv)
                    img_hsv.save(new_path)
                    print(f'Converted {file} to HSV')

                    # Hitung nilai modus hue
                    hue_values = np.array(img_hsv)[:, :, 0]
                    mode_hue = find_mode(hue_values)
                    print(f'Mode hue value for {file}: {mode_hue}')
                    mode_hue_values.append(mode_hue)
                    labels.append(label)
except Exception as e:
    print(f"Error processing images: {e}")

print(f'Total processed images: {len(mode_hue_values)}')
print(f'Labels: {labels}')

# Menggunakan Decision Tree
if len(mode_hue_values) > 0:
    label_mapping = {'unripe': 0, 'ripe': 1, 'overripe': 2, 'rotten': 3}
    Y = np.array([label_mapping[label] for label in labels])
    X = np.array(mode_hue_values).reshape(-1, 1)
    DT_model = DecisionTreeClassifier()
    DT_model.fit(X, Y)

    # Prediksi contoh data baru
    data_baru = [mode_hue_values[0]]  # Ganti dengan data baru yang ingin diprediksi
    hasilprediksi = DT_model.predict(np.array(data_baru).reshape(-1, 1))

    # Kondisi Kematangan
    if hasilprediksi == 0:
        print('Pisang ini belum matang')
    elif hasilprediksi == 1:
        print('Pisang ini matang')
    elif hasilprediksi == 2:
        print('Pisang ini terlalu matang')
    elif hasilprediksi == 3:
        print('Pisang ini busuk')
    else:
        print('Pisang tidak teridentifikasi')

    # Menghitung akurasi menggunakan confusion matrix
    Y_pred = DT_model.predict(X)
    cm = confusion_matrix(Y, Y_pred)
    print('Confusion Matrix:')
    print(cm)

    # Menghitung akurasi
    accuracy = np.trace(cm) / np.sum(cm) * 100
    print(f'Akurasi: {accuracy:.2f}%')
else:
    print("No images were processed. Please check the dataset path and ensure images are available.")